<h1>Capstone Project - The Battle of Neighborhoods</h1>

<h2>2. Data acquisition and cleaning</h2>

<h3>2.1 Data sources</h3>
<p>Data sources for this project will be mostly same as I've used during Module 3. of the course.</p>
<p><b>New York</b> has a total of 5 boroughs and 306 neighborhoods. In order to segement the neighborhoods and explore them, we will essentially need a dataset that contains the 5 boroughs and the neighborhoods that exist in each borough, as well as the the latitude and logitude coordinates of each neighborhood.
I will use dataset provided in the course: <a target="_blank" rel="noreferrer" href="https://geo.nyu.edu/catalog/nyu_2451_34572">https://geo.nyu.edu/catalog/nyu_2451_34572</a>. Data file: <a target="_blank" rel="noreferrer" href="https://cocl.us/new_york_dataset">https://cocl.us/new_york_dataset</a>
</p>
<p>For <b>Toronto</b> I will scrape the following Wikipedia page, <a target="_blank" rel="noreferrer" href="https://en.wikipedia.org/wiki/List_of_postal_codes_of_Canada:_M">List_of_postal_codes_of_Canada:_</a> to gather data about boroughs and neighborhoods. Wiki page doesn't contain coridates for neighborhoods. I could use geocoder library to gather these data, but becase of the limitations, probably I will use location data from course <a target="_blank" rel="noreferrer" href="https://cocl.us/Geospatial_data">Geospatial data for Toronto neighborhoods</a></p>
<p>For the geograpical coordinate of the cities I can use geocoder library.</p>
<p>To explore the neighborhoods and segment them, I will be utilizing the Foursquare API (https://foursquare.com). For each neighborhood I will be gathering data about venues. Data are in JSON format.</p>



<h3>2.2 Data cleaning</h3>
<p>For New York neighborhoods data are in JSON format. I need to transform them into table with required data.</p>
<p>I need to extract:</p>
<ul>
    <li><b>Borough</b> - name of the city borough, where neighborhood belongs, see properties.borough, e.g. 'Queens'
    <li><b>Neighborhood</b> - name of the neighborhood, see properties.name, e.g. 'Lefrak City'
    <li><b>Latitude</b> - latitude of the neighborhood, see geometry.coordinates second array element, e.g. '40.736074570830795'
    <li><b>Longitude</b> - longitude of the neighborhood, see geometry.coordinates first array element, e.g. '-73.8625247141374'
</ul>
<p>Example for 'Lefrak City'</p>
<pre>
    {'type': 'Feature',
          'id': 'nyu_2451_34572.190',
          'geometry': {'type': 'Point',
           'coordinates': [-73.8625247141374, 40.736074570830795]},
          'geometry_name': 'geom',
          'properties': {'name': 'Lefrak City',
           'stacked': 2,
           'annoline1': 'Lefrak',
           'annoline2': 'City',
           'annoline3': None,
           'annoangle': 0.0,
           'borough': 'Queens',
           'bbox': [-73.8625247141374,
            40.736074570830795,
            -73.8625247141374,
            40.736074570830795]}
    }
</pre>

<p>Foursquare API responses about venues are in JSON format. I need to transform them into table with required data. Request to Foursquare API will be performed for each neighborhood, based on coordinates. This way I can join neighborhood data with venues data.</p>
<p>I will extract:</p>
<ul>
    <li><b>Venue id</b> - id of the venue, see venue.id, e.g. '4baf59e8f964a520a6f93be3', it could be useful to remove duplicates, if range in query is to big
    <li><b>Venue distance</b> - how far the venue is from point used in request (center of neighborhood), see venue.location.distance, e.g. 376, it could be useful to remove duplicates, if range in query is to big
    <li><b>Venue Name</b> - name of the venue, see venue.name, e.g. 'Queens'
    <li><b>Venue Category</b> - name of the venue category, the <b>most</b> important information, will be used for clustering, see venue.categories.name, e.g. 'Yoga Studio'
    <li><b>Venue Latitude</b> - latitude of the venue, see venue.location.lat, e.g. '40.876843690797934'
    <li><b>Venue Longitude</b> - longitude of the venue, see venue.location.lat, e.g. '-73.90620384419528'
</ul>
<p>Probably I can later remove most of data gathered about venues, since clustering will be based on venues categories and how common there are for the selected neighborhood.</p>
<p>Example data about venue from explore API call</p>
<pre>
            {'reasons': {'count': 0,
              'items': [{'summary': 'This spot is popular',
                'type': 'general',
                'reasonName': 'globalInteractionReason'}]},
             'venue': {'id': '4baf59e8f964a520a6f93be3',
              'name': 'Bikram Yoga',
              'location': {'address': '5500 Broadway',
               'crossStreet': '230th Street',
               'lat': 40.876843690797934,
               'lng': -73.90620384419528,
               'labeledLatLngs': [{'label': 'display',
                 'lat': 40.876843690797934,
                 'lng': -73.90620384419528}],
               'distance': 376,
               'postalCode': '10463',
               'cc': 'US',
               'city': 'Bronx',
               'state': 'NY',
               'country': 'United States',
               'formattedAddress': ['5500 Broadway (230th Street)',
                'Bronx, NY 10463',
                'United States']},
              'categories': [{'id': '4bf58dd8d48988d102941735',
                'name': 'Yoga Studio',
                'pluralName': 'Yoga Studios',
                'shortName': 'Yoga Studio',
                'icon': {'prefix': 'https://ss3.4sqi.net/img/categories_v2/shops/gym_yogastudio_',
                 'suffix': '.png'},
                'primary': True}],
              'photos': {'count': 0, 'groups': []}},
             'referralId': 'e-0-4baf59e8f964a520a6f93be3-1'}
</pre>
<p>For Toronto neighborhoods data are placed on table on wiki. I need to scrap the HTML page and transform them into table with data below.</p>
<p>I need to extract:</p>
<ul>
    <li><b>Postcode</b> - post code for neighborhood, e.g. 'M5H'
    <li><b>Borough</b> - name of the city borough, where neighborhood belongs, e.g. 'Downtown Toronto'
    <li><b>Neighborhood</b> - name of the neighborhood, e.g. 'Adelide'
</ul>
<p>Then I will group gathered data by Postcode, I will remove not assigned post codes and merge such data with cordiates data, by using post code. Rename of column is required.</p>
<p>Here is description of coordinates data for each post code of Toronto. Data are in CSV format so are easy to use.</p>
<ul>
    <li><b>Postal Code</b> - post code for neighborhood, e.g. 'M1B'
    <li><b>Latitude</b> - latitude of the neighborhood, e.g. '43.8066863'
    <li><b>Longitude</b> - longitude of the neighborhood, e.g. '-79.1943534'
</ul>
<p>After join will take place, I could drop 'Postcode' column.<p>
<p>To be able to join data about neighborhoods for both cities, and have a way to distinguish them, I need to add new column <b>City</b> with respectively 'Toronto' and 'New York' values.</p>
<p>After join I will have dataset with following columns</p>
<ul>
    <li><b>City</b>
    <li><b>Borough</b>
    <li><b>Neighborhood</b>
    <li><b>Latitude</b>
    <li><b>Longitude</b>
</ul>


<h3>2.3 Feature selection</h3>
<p>For clustering, I will try to use popular venues categories for each neighborhoods, see 'Venue Category' above, but I'm not sure if this will be sufficent.</p>
<p>I can group results base on Venue Category, calculate how common they are in particular neighborhood surroundings, and perform clustering base on these data.</p>
<p>Using segmentation and clustering, I hope I can determine the similarity or dissimilarirty of both cities and they neighborhoods, and properly name clustered areas located inside the city.</p>


